# Task 3-1
Нужно написать скрипт, который скачивает все данные прошедших президентских выборов для всех избирательных участков.

Входная точка по ссылке. Затем нужно перейти на сайты региональных избирательных комиссий. Результаты нужно сохранить в cvs-файл, sqlite базе данных или parquet-файле. В итоге должна получиться таблица с полями: - название региона - название ТИК - номер УИК - 20 стандартных полей из итогового протокола

In [1]:
from multiprocessing import Pool as process_pool
from pyquery import PyQuery as pq
from urllib.parse import parse_qs
from typing import List, Dict
from toolz import partial
import requests
import pandas as pd
import tqdm
ENTRY_POINT_URL = 'http://notelections.online/region/region/izbirkom?action=show&root=1&tvd=100100084849066&vrn=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100084849066&type=227'
API_JSON_URL = "http://notelections.online/region/izbirkom?action=tvdTree&tvdchildren=true&vrn={}&tvd={}"
BASE_URL = 'http://notelections.online/'
VRN = 'vrn'
TVD = 'tvd'

In [2]:
query_params = parse_qs(ENTRY_POINT_URL)
areas_links = requests.get(API_JSON_URL.format(query_params[VRN][0], query_params[TVD][0])).json()

In [3]:
def link_iterate(links: List[Dict], acc: List[Dict] = []) -> List[Dict]:
    if len(links) == 0:
        return acc
    link, *next_step = links
    query_params = parse_qs(link['href'])
    generate_link = API_JSON_URL.format(query_params[VRN][0], query_params[TVD][0]) 
    link |= {'children': requests.get(generate_link).json()}
    return link_iterate(next_step, acc+[link])

all_links = link_iterate(areas_links)

In [15]:
def get_multindex_columns(*columns: List[str]):
    return pd.MultiIndex.from_product(columns, names=['Регион','Район','УИК'])
    

def get_table(link: Dict, name: str) -> pd.DataFrame:
    url_link = BASE_URL + link['href'] + '&type=227&report_mode=null'
    area_page = pq(url=url_link)
    tables = area_page('table')
    html_table, = tables.filter(lambda: 'Сумма' in pq(this).text()).items()
    df, = pd.read_html('<table>'+html_table.html()+'<\table>', index_col=[1])
    df = df.drop('Unnamed: 0', axis='columns')
    df.columns = get_multindex_columns([name], [link['text']], df.columns)
    return df.T

In [16]:
def get_area_table(link):
    area_df = pd.concat(process_pool(8).map(partial(get_table, name=link['text']), link['children']))
    return area_df

res = [get_area_table(link) for link in tqdm.tqdm(all_links[:-2])]



  0%|          | 0/85 [00:00<?, ?it/s]

  1%|          | 1/85 [00:02<03:15,  2.33s/it]

  2%|▏         | 2/85 [00:06<04:43,  3.42s/it]

  4%|▎         | 3/85 [00:29<16:49, 12.32s/it]

  5%|▍         | 4/85 [00:37<14:27, 10.71s/it]

  6%|▌         | 5/85 [00:55<17:56, 13.46s/it]

  7%|▋         | 6/85 [00:58<13:00,  9.89s/it]

  8%|▊         | 7/85 [01:04<10:59,  8.46s/it]

  9%|▉         | 8/85 [01:10<09:45,  7.60s/it]

 11%|█         | 9/85 [01:15<08:33,  6.76s/it]

 12%|█▏        | 10/85 [01:22<08:36,  6.89s/it]

 13%|█▎        | 11/85 [01:29<08:40,  7.03s/it]

 14%|█▍        | 12/85 [01:38<09:15,  7.61s/it]

 15%|█▌        | 13/85 [01:44<08:39,  7.22s/it]

 16%|█▋        | 14/85 [01:53<08:51,  7.48s/it]

 18%|█▊        | 15/85 [02:03<09:56,  8.52s/it]

 19%|█▉        | 16/85 [02:08<08:29,  7.38s/it]

 20%|██        | 17/85 [02:31<13:33, 11.97s/it]

 21%|██        | 18/85 [02:38<11:44, 10.51s/it]

 98%|█████████▊| 85/87 [1:12:06<01:41, 50.90s/it]A


 24%|██▎       | 20/85 [02:55<09

In [20]:
*_, bad_link1, bad_link2 = all_links
df = pd.concat(res+[get_table(bad_link1, bad_link1['text']), get_table(bad_link2, bad_link2['text'])])